# Einführung in Neuronale Netze

<img src="https://upload.wikimedia.org/wikipedia/commons/3/3d/Neural_network.svg" alt="Ein Neuronales Netz (vereinfacht)" width="400px" />

## EINLEITUNGSTEXT
...

## Das einzelne Neuron
<img src="https://miro.medium.com/max/856/1*O7YSSqlOdQuNgMrH7J_3dg.png" alt="Das einzelne Neuron" />

Ein Neuronales Netz besteht aus vielen Neuronen, die miteinander verbunden sind. Dabei sind die Neuronen denen im Gehirn nachempfunden, welche untereinander verbunden sind und die aufeinander reagieren. Bei künstlichen Neuronalen Netzen hat jedes Neuron eine oder mehrere Eingaben $x_1...x_m$ und eine Ausgabe $o_j$. Die Eingaben werden mit Gewichten $w_1...w_m$ multipliziert und anschließend addiert ($ net_j = \displaystyle\sum_{i=1}^{m} (x_i \cdot w_i) $). Manchmal gibt es noch einen Bias $b_j$ ($j$ steht für ein bestimmtes Neuron. Jedes Neuron hat einen eigenen Bias). Dieser wird auf diese Summe drauf addiert ($ net_j = \displaystyle\sum_{i=1}^{m} (x_i \cdot w_i) + b_j $).Das Ergebnis wird dann in eine Aktivierungsfunktion $\varphi_j(x)$  gegeben, welche das Ergebnis in die Ausgabe des Neurons umwandelt $o_j = \varphi_j(net_j)$ und die dann ein Eingabewert (ein anderes $x$) für die nächsten Neuronen ist (oder die Ausgabe des Neuronalen Netzes darstellt, wenn das Neuron an der hintersten Ebene ist).

### Ein einfaches Beispiel 
 
Wir nehmen an wir haben ein Neuron mit zwei Inputs, welches die Sigmoid Funktion als Aktivierungsfunktion verwendet. Die folgenden Parameter sind gegeben:

$ w = [0, 1] == w1 = 0\ und\ w2 = 1 \\
$ b = 4

Nun geben wir dem Neuron ein input x mit x = [2, 3]

$$
(w*x)+b = ((w1 * x1) + (w2 * x2)) + b 
        = 0 * 2 + 1 * 3 + 4
        = 7
y = f(w * x + b) = f(7) = 0.999       
$$  

Für den Input x = [2, 3] bekommen wir einen Output von 0,999. Dieser Prozess in welchem man inputs eingibt, um ein Output zu bekommen ist bekannt als **feedforward** 

## Ein Neuron programmieren

Um das Neuron zu programmieren importieren wir Numpy. Numpy ist eine beliebte Python Library um mit Zahlen zu rechnen. 

In [2]:
#Wir importieren numpy als np um numpy nicht immer ausschreiben zu müssen
import numpy as np


def sigmoid(x):
  # Unsere Aktivierungsfunktion: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def feedforward(self, inputs):
    # Gewichtsinputs, Addiere Gewichte und dann nutze die Aktivierungsfunktion
    # np.dot ist eine Funktion um zwei Arrays zu multiplizieren 
    total = np.dot(self.weights, inputs) + self.bias
    return sigmoid(total)

weights = np.array([0, 1]) # w1 = 0, w2 = 1
bias = 4                   # b = 4
n = Neuron(weights, bias)

x = np.array([2, 3])       # x1 = 2, x2 = 3
print(n.feedforward(x))    # 0.9990889488055994

0.9990889488055994


Wie man sieht, kommen wir mit unserem selbstgecodeten Neuron auf dasselbe Ergebnis, wie zuvor nur mit Mathe.

## Überblick über die Aktivierungsfunktionen

### Warum gibt es Aktivierungsfunktionen?
 Aktivierungsfunktionen werden verwendet, um dem neuronalen Netzwerk eine nicht lineare Eigenschaft zu verleihen. Auf diese Weise kann das Netz komplexere Beziehungen und Muster in den Daten modellieren. 
### Beispiele für Aktivierungsfunktionen
<img src="https://assets-global.website-files.com/5d7b77b063a9066d83e1209c/60d244bb0e12c94fb442c01e_pasted%20image%200%20(4).jpg" alt="Sigmoid-Aktiverungsfunktion" height="300px" />
<img src="https://assets-global.website-files.com/5d7b77b063a9066d83e1209c/60d24547f85f71e3bd2339f8_pasted%20image%200%20(5).jpg" alt="Sigmoid-Aktiverungsfunktion" height="300px" />
<img src="https://assets-global.website-files.com/5d7b77b063a9066d83e1209c/60d24d1ac2cc1ded69730feb_relu.jpg" alt="ReLu-Aktiverungsfunktion" height="300px" />

noch beschreibenden Text einfügen...

## Ein künstliches Neuronales Netz
<img src="https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_01_03-DeepNeuralNetwork-WHITEBG.png" alt="Ein neuronales Netz" width="400px" />

### Aufbau eines neuronalen Netzes

Ein neuronales Netz besteht immer aus mindestens einer Schicht mit jeweils mindestens einem Neuron. Die zu verarbeitenden Daten werden in die erste Schicht neuronen gegeben (Input-Layer), wird dann durch das Netz verarbeitet und die letzte Schicht bildet ein oder mehr Neuronen die Ausgabe (die Anzahl von Neuronen pro Schicht - vor allem bei der Ein- und Ausgabeschicht - wird meistens vom Anwendungsfall bestimmt). Meistens sind alle Neuronen einer Schicht mit den Neuronen der nächsten Schicht verknüpft, d.h. die Ausgaben aller Neuronen einer Schicht sind Eingaben für die Neuronen der nächsten Schicht (das ist aber nicht zwingend so!). Bevor die Eingabewerte jedoch addiert werden und ihren Weg durch die Aktivierungsfunktion gehen, wird jede Ausgabe eines Neurons zu einem anderen mit einem eigenen Gewicht multipliziert. Dabei hat jede Verbindung ein eigenes Gewicht. Die Gewichte werden beim Training des neuronalen Netzes angepasst, sodass das Netz besser lernt, und eben diese Gewichte gilt es so zu wählen, dass das neuronale Netz die besten Ausgaben gibt.